In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
import google.generativeai as genai
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

In [ ]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=768, metric="cosine", spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  ),
  deletion_protection="disabled"
)

In [18]:
import json 
data = json.load(open("reviews.json"))

data["reviews"]



[{'state': 'California',
  'city': 'Los Angeles',
  'stars': 5,
  'review': 'A vibrant city with endless entertainment and fantastic weather!'},
 {'state': 'New York',
  'city': 'New York City',
  'stars': 4,
  'review': 'Great for nightlife and culture, but traffic is a nightmare.'},
 {'state': 'Texas',
  'city': 'Austin',
  'stars': 5,
  'review': 'A fun, music-filled city with amazing food and welcoming people.'},
 {'state': 'Illinois',
  'city': 'Chicago',
  'stars': 4,
  'review': 'Beautiful architecture and deep-dish pizza, but winters are harsh.'},
 {'state': 'Florida',
  'city': 'Miami',
  'stars': 3,
  'review': "Lovely beaches, but it's too hot and humid most of the year."},
 {'state': 'Nevada',
  'city': 'Las Vegas',
  'stars': 4,
  'review': 'Perfect for a quick getaway, though it can get overwhelming.'},
 {'state': 'Colorado',
  'city': 'Denver',
  'stars': 5,
  'review': 'Great for outdoor enthusiasts, with stunning views and friendly locals.'},
 {'state': 'Washington',
 

In [32]:
#create embeddings

genai.configure(api_key=os.getenv("GEMINIAI_API_KEY"))
processed_data = []
client = genai

# Assuming 'client' is already instantiated and correctly configured
for review in data['reviews']:
    # Ensure the review text is being passed correctly
    model = "models/text-embedding-004"
    response = genai.embed_content(
                                model=model,
                                content=review['review'],
                                #task_type="retrieval_document",
                                )
    # response = client.generate_embeddings(
    #     input=review['review'],
    #     model="text-embedding-005",

    # )

    print(response)
    #embedding = response.data[0].embedding 
    
    # Assuming the response contains the embeddings
    
    # # Process or store the embeddings as needed


    # #individual piece of data
    processed_data.append({
        "values": response['embedding'],
        "id": review["city"],
        "metadata": {
            "review": review["review"],
            "state": review["state"],
            "stars" : review["stars"]
        }
    })

    # for review in data['review']:
    #     response = client.embeddings.create(
    #         input=review['review'],
    #         model="text-embedding-3-small",
    #     )


{'embedding': [-0.0015006175, 0.011338981, 0.038019978, 0.0050780396, 0.0067466646, -0.052020676, 0.00079473, -0.02144273, 0.028515698, 0.05014162, -0.05492301, 0.05010295, 0.02082227, -0.010123367, 0.03807996, -0.022801172, -0.0471117, 0.014552404, -0.15191309, -0.038748544, 0.023211718, -0.0009474588, 0.0730699, -0.051709313, 0.023414392, -0.021215925, 0.0123841725, 0.017970411, 0.033031214, 0.036283955, -0.016841326, 0.04066232, 0.0056357277, -0.013797509, -0.03638691, 0.004581084, 0.027998919, -0.07366585, 0.022236187, 0.0016244107, -0.085761204, 0.03354654, -0.02503103, -0.024930825, -0.04572523, -0.07046531, -0.023535566, 0.022041177, 0.013015534, 0.042669058, 0.016743833, 0.012892009, 0.004989652, 0.03410962, -0.018607266, -0.01902047, -0.05642336, 0.0149620995, 0.11964834, 0.013683578, 0.027470585, 0.005106622, 0.01632254, -0.019074716, 0.043895733, -0.054455888, -0.04429278, 0.04865147, 0.02387023, -0.009263117, -0.013807383, 0.04779293, -0.008933326, 0.044666134, 0.03330619, 

In [33]:
processed_data[0]

{'values': [-0.0015006175,
  0.011338981,
  0.038019978,
  0.0050780396,
  0.0067466646,
  -0.052020676,
  0.00079473,
  -0.02144273,
  0.028515698,
  0.05014162,
  -0.05492301,
  0.05010295,
  0.02082227,
  -0.010123367,
  0.03807996,
  -0.022801172,
  -0.0471117,
  0.014552404,
  -0.15191309,
  -0.038748544,
  0.023211718,
  -0.0009474588,
  0.0730699,
  -0.051709313,
  0.023414392,
  -0.021215925,
  0.0123841725,
  0.017970411,
  0.033031214,
  0.036283955,
  -0.016841326,
  0.04066232,
  0.0056357277,
  -0.013797509,
  -0.03638691,
  0.004581084,
  0.027998919,
  -0.07366585,
  0.022236187,
  0.0016244107,
  -0.085761204,
  0.03354654,
  -0.02503103,
  -0.024930825,
  -0.04572523,
  -0.07046531,
  -0.023535566,
  0.022041177,
  0.013015534,
  0.042669058,
  0.016743833,
  0.012892009,
  0.004989652,
  0.03410962,
  -0.018607266,
  -0.01902047,
  -0.05642336,
  0.0149620995,
  0.11964834,
  0.013683578,
  0.027470585,
  0.005106622,
  0.01632254,
  -0.019074716,
  0.043895733,
  -0.

In [30]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"

)

upserted_count: 40

In [31]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 40}},
 'total_vector_count': 40}